In [1]:
import pandas as pd
import os
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Alignment, Font
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.cell import MergedCell
from openpyxl.utils import get_column_letter

In [2]:
def list_files(directory):
    paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            paths.append(os.path.join(root, file))
    return paths

In [3]:
results_directory = "/home/aleksandr/Desktop/WORK/OLINK_suicide_PSY_project/Data_preprocessing_analysis/"
all_files_results = list_files(results_directory)
all_files_results

['/home/aleksandr/Desktop/WORK/OLINK_suicide_PSY_project/Data_preprocessing_analysis/RNA_seq_preproc_STAR.md',
 '/home/aleksandr/Desktop/WORK/OLINK_suicide_PSY_project/Data_preprocessing_analysis/SubreadUsersGuide.pdf',
 '/home/aleksandr/Desktop/WORK/OLINK_suicide_PSY_project/Data_preprocessing_analysis/RNAseq_gene_probes.csv',
 '/home/aleksandr/Desktop/WORK/OLINK_suicide_PSY_project/Data_preprocessing_analysis/mapping_affy_hugene_2_0_st_v1_enseml.txt',
 '/home/aleksandr/Desktop/WORK/OLINK_suicide_PSY_project/Data_preprocessing_analysis/GSE247998_SraRunTable.txt',
 '/home/aleksandr/Desktop/WORK/OLINK_suicide_PSY_project/Data_preprocessing_analysis/fastqc_v0.12.1.zip',
 '/home/aleksandr/Desktop/WORK/OLINK_suicide_PSY_project/Data_preprocessing_analysis/mapping_affy_hg_u133a_2_enseml.txt',
 '/home/aleksandr/Desktop/WORK/OLINK_suicide_PSY_project/Data_preprocessing_analysis/GSE247998_SRR_ids_batch_2.txt',
 '/home/aleksandr/Desktop/WORK/OLINK_suicide_PSY_project/Data_preprocessing_analysis

In [4]:
selected_files = [x for x in all_files_results if "_demographics.xlsx" in x]

In [5]:
selected_files

['/home/aleksandr/Desktop/WORK/OLINK_suicide_PSY_project/Data_preprocessing_analysis/GSE92538_U133_PLUS2_results/GSE92538_PLUS2_demographics.xlsx',
 '/home/aleksandr/Desktop/WORK/OLINK_suicide_PSY_project/Data_preprocessing_analysis/GSE92538_U133A_results/GSE92538_U133A_demographics.xlsx',
 '/home/aleksandr/Desktop/WORK/OLINK_suicide_PSY_project/Data_preprocessing_analysis/GSE208338_results/GSE208338_demographics.xlsx',
 '/home/aleksandr/Desktop/WORK/OLINK_suicide_PSY_project/Data_preprocessing_analysis/GSE243356_results/GSE243356_demographics.xlsx',
 '/home/aleksandr/Desktop/WORK/OLINK_suicide_PSY_project/Data_preprocessing_analysis/GSE5389_results/GSE5389_demographics.xlsx',
 '/home/aleksandr/Desktop/WORK/OLINK_suicide_PSY_project/Data_preprocessing_analysis/GSE101521_results/GSE101521_demographics.xlsx',
 '/home/aleksandr/Desktop/WORK/OLINK_suicide_PSY_project/Data_preprocessing_analysis/GSE248260_results/GSE248260_demographics.xlsx',
 '/home/aleksandr/Desktop/WORK/OLINK_suicide_PSY

In [6]:
file_names = [os.path.basename(x).replace(".xlsx", "") for x in selected_files]

In [7]:
datasets = [pd.read_excel(x) for x in selected_files]


Prompt to ChatGPT
"
I have a list of pandas dataframes in python called datasets. I would like you to write code that will save each of these datasets in the excel file. You must perform these steps:



-Each table should be on a separate sheet. 
-IF the row in the table contains pattern "GSE", the cells in this row must be merged and the font set to bold
-IF the row in the table contains pattern "Dataset includes", the cells in this row must be merged
-The header of tables should be removed. 
-The first column of every table should be formatted in bold font. 
-The text in every cell has to be center-aligned.
-Width of every cell is set to 100
-Font must be set to Times New Roman
-Output file is called "Data_S2_combined_demogr.xlsx"

"

In [8]:
# Create a new workbook
wb = Workbook()

# Remove the default sheet
wb.remove(wb.active)

# Iterate over the list of dataframes and create a new sheet for each
for idx, df in enumerate(datasets):
    # Create a new sheet
    ws = wb.create_sheet(title=file_names[idx])
    
    # Remove the header and convert the DataFrame to a list of rows
    rows = list(dataframe_to_rows(df, index=False, header=False))
    
    # Write the rows to the worksheet
    for r_idx, row in enumerate(rows, 1):
        ws.append(row)
        
        # Apply font and alignment settings
        for c_idx, cell in enumerate(row, 1):
            cell_ref = ws.cell(row=r_idx, column=c_idx)
            cell_ref.alignment = Alignment(horizontal='center', vertical='center')
            cell_ref.font = Font(name='Times New Roman')
            
            # Format the first column in bold
            if c_idx == 1:
                cell_ref.font = Font(name='Times New Roman', bold=True)

        # Check for the special patterns
        if any("GSE" in str(cell) for cell in row):
            ws.merge_cells(start_row=r_idx, start_column=1, end_row=r_idx, end_column=len(row))
            ws[r_idx][0].font = Font(name='Times New Roman', bold=True)

        if any("Dataset includes" in str(cell) for cell in row):
            ws.merge_cells(start_row=r_idx, start_column=1, end_row=r_idx, end_column=len(row))


    # Adjust row height to fit the text
    for row in ws.iter_rows():
        ws.row_dimensions[row[0].row].height = 70

        # Set the width of all columns
    for col in ws.columns:
        max_length = 50
        col_letter = get_column_letter(col[0].column)
        ws.column_dimensions[col_letter].width = max_length

# Save the workbook
wb.save("Data_S3_combined_demogr.xlsx")